In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import json
import DownArrow as dn

In [5]:
G = nx.complete_graph(6) # choose graph

P = dn.Graph_Ideal(G, nx.DiGraph(), 1) #generate the ideal

#diagnostic graphs = nx.get_node_attributes(P, "graph")

positions = dn.calculate_positions(P)

dn.build_and_print_json(P) #generates the json file for the ideal


#nx.draw_networkx(P, pos=nx.spring_layout(P), with_labels=True)

In [ ]:
#run this cell for a preview drawing
plt.figure(figsize=(20, 30))
nx.draw_networkx(P, pos=positions, with_labels=True, node_size=700, node_color="lightblue", font_size=10, font_weight="bold")
plt.show()

In [9]:
json_data = {'vertices': {}, 'edges': list(P.edges())}
pos = calculate_positions(P)
for node, data in P.nodes(data=True):
    nested_graph = data['graph']
    json_data['vertices'][node] = {
        'position': {
            'x': pos[node][0],
            'y': pos[node][1]
        },
        'associated_graph': graph_to_json_structure(nested_graph)
    }

json_str = json.dumps(json_data, indent=2)
with open('graph_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)